In [ ]:
import json
import pandas as pd
with open("outputs/buckets_07-29_charsim17.json","r") as file:
  buckets = json.load(file)
bucket_map={}
for name in buckets:
  for action in buckets[name]:
    bucket_map[action]=name
common_bucket_names = [name for name in buckets.keys() if len(buckets[name])>=10]
common_bucket_names_inv={name:i for i,name in enumerate(common_bucket_names)}
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
for i,df in enumerate(history_df_by_term):
  df["term"]=i+111
history_df=pd.concat(history_df_by_term)
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
bills={bill_id:group for (bill_id,group) in history_df.groupby("bill_id")}
bill_ids = list(bills.keys())

MIN_TERM=min(history_df["term"])
MAX_TERM=max(history_df["term"])
N_TERMS=MAX_TERM-MIN_TERM+1


In [ ]:
print(len(buckets.keys()))

In [ ]:
print(len(common_bucket_names))

In [ ]:
import numpy as np

alpha=2/3
data=[]

bill_id=history_df.iloc[0]["bill_id"]
predecessors_vec=np.zeros(len(common_bucket_names))
predecessor_vec=np.zeros(len(common_bucket_names))
one_hot_curr = np.zeros(len(common_bucket_names))
inputs=[]
outputs=[]
n_predecessors_prev=0
for i,row in history_df.iterrows():
  one_hot_prev = np.array(one_hot_curr)
  one_hot_curr = np.zeros(len(common_bucket_names))
  output_vec = np.zeros(len(common_bucket_names)+1)
  if row.bucket in common_bucket_names_inv:
    one_hot_curr[common_bucket_names_inv[row.bucket]]=1
    output_vec[common_bucket_names_inv[row.bucket]]=1
  else:
    output_vec[-1]=1
  
  chamber = np.zeros(2)
  if row["chamber"]=="House":
    chamber[0]=1
  if row["chamber"]=="Senate":
    chamber[1]=1

  # if sum(one_hot_prev)!=1 and sum(one_hot_prev)!=0:
  #   print("non-0/1 sum",i)
  # n_predecessors=np.sum(predecessors_vec)
  # if n_predecessors!=0 and n_predecessors!=n_predecessors_prev+1 and n_predecessors!=n_predecessors_prev:
  #   print(n_predecessors,n_predecessors_prev,i)
  # n_predecessors_prev=n_predecessors
  if row["bill_id"]!=bill_id:
    bill_id=row["bill_id"]
    one_hot_prev=np.zeros(len(common_bucket_names))
    predecessors_vec=np.zeros(len(common_bucket_names))
    predecessor_vec=np.zeros(len(common_bucket_names))
  predecessor_vec=(1-alpha)*one_hot_prev+alpha*np.array(predecessor_vec)
  predecessors_vec=np.array(predecessors_vec)+one_hot_prev
  one_hot_term=np.zeros(N_TERMS)
  one_hot_term[row.term-MIN_TERM]=1
  # input_vec = np.concat([predecessor_vec,predecessors_vec,one_hot_term])
  entry={"predecessor":predecessor_vec,"predecessors":predecessors_vec,"term":one_hot_term,"chamber":chamber,"output":output_vec}
  # print(entry)
  data.append(entry)

# print(data[0])
# inputs=np.stack(inputs)
# outputs=np.stack(outputs)
# np.savetxt("outputs/vectors/07-29_charsim17/input",inputs[0])
# np.savetxt("outputs/vectors/07-29_charsim17/output",outputs)


In [ ]:
display(data[0])

In [ ]:
np.savez_compressed("outputs/prediction_vecs_08-04_charsim17.npz",data)

In [ ]:
import numpy as np
data = np.load("outputs/prediction_vecs_08-04_charsim17.npz",allow_pickle=True)["arr_0"]
display(data[0])
# display(outs)

In [ ]:
import sklearn.preprocessing
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sklearn

class ActionDataset(Dataset):
  def __init__(self,path):
    data = np.load(path,allow_pickle=True)["arr_0"]
    self.inputs = [np.concatenate([entry["predecessor"],entry["predecessors"],entry["term"],entry["chamber"]]) for entry in data]
    self.inputs = sklearn.preprocessing.StandardScaler().fit_transform(self.inputs)
    self.outputs = [entry["output"] for output in data]
  def __len__(self):
    return len(self.inputs)
  def __getitem__(self,idx):
    return self.inputs[idx],self.outputs[idx]

ds = ActionDataset("outputs/prediction_vecs_08-04_charsim17.npz")
# print(len(ds))
train_dataset,test_dataset,val_dataset = torch.utils.data.random_split(ds,[0.6,0.2,0.2])

In [ ]:
def print_arr(arr):
  for line in arr:
    print([float(x) for x in line])

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)

# class ActionModel(torch.nn.Module):
#   def __init__(self,input_len,output_len):
#     self.linear= torch.nn.Linear
#     self.input_len=input_len
#     self.output_len = output_len
#   def forward(self,input):

model = torch.nn.Linear(len(common_bucket_names)*2+N_TERMS+2,len(common_bucket_names)+1)
# model.load_state_dict(torch.load("outputs/models/08-04_lr1e-5_beta.3/epoch85.pt"))
optim = torch.optim.Adam(model.parameters(),lr=1e-5)
pred_loss_fn = torch.nn.CrossEntropyLoss()
beta=0.1
folder = "outputs/models/08-04_lr1e-5_beta.1"
if not os.path.isdir(folder):
    os.mkdir(folder)

for epoch in range(100):
    for i,(input,output) in enumerate(train_loader):
        optim.zero_grad()
        input=input.float()
        output=output.float()
        pred = model(input)
        pred_loss = pred_loss_fn(pred,output)
        ridge_loss = torch.sum(torch.abs(model.weight))
        loss = pred_loss+beta*ridge_loss
        loss.backward()
        optim.step()
    print(f"finished epoch {epoch}")

    torch.save({"model":model.state_dict(),"optim":optim.state_dict()},folder+f"/epoch{epoch}.pt")
    with torch.no_grad():
        val_pred_loss=torch.scalar_tensor(0)
        val_ridge_loss=torch.sum(torch.abs(model.weight))
        for input,output in val_loader:
            input=input.float()
            output=output.float()
            pred = model(input)
            # for line in output:
            #   for i,x in enumerate(line):
            #     if x>=0.9:
            #       print(i)
            #   print(torch.sum(line))
            # break
            # print_arr(output.numpy())
            # print("sum:",torch.sum(output))
            val_pred_loss+=pred_loss_fn(pred,output)
        val_pred_loss/=len(val_loader)
    with open(folder+f"/loss","a") as file:
        file.write(f"epoch {epoch}. val pred loss:{val_pred_loss}. val ridge loss:{val_ridge_loss}")
        file.write("\n")
    print("val pred loss:",val_pred_loss,"val ridge loss:",val_ridge_loss)



In [ ]:
display(model.weight.detach().numpy()[:,len(common_bucket_names)+1])

In [ ]:

model = torch.nn.Linear(len(common_bucket_names)*2+N_TERMS+2,len(common_bucket_names)+1)
model.load_state_dict(torch.load("outputs/models/08-04_lr1e-5_beta.1/epoch85.pt"))
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)
with torch.no_grad():
  val_pred_loss=torch.scalar_tensor(0)
  val_ridge_loss=torch.sum(torch.abs(model.weight))
  for input,output in val_loader:
    input=input.float()
    output=output.float()
    pred = model(input)
    for line in pred:
      print(list(line))
      # for i,x in enumerate(line):
      #   if x>=0.1:
      #     print(i,float(x),end="")
      # print()
      print(torch.sum(line))
    # break
    # print_arr(output.numpy())
    # print("sum:",torch.sum(output))
  # val_pred_loss/=len(val_loader)
  print("val pred loss:",val_pred_loss,"val ridge loss:",val_ridge_loss)

In [ ]:
from collections import defaultdict
model.load_state_dict(torch.load("outputs/models/08-04_lr1e-5_beta.1/epoch85.pt"))
weights = model.weight.detach()
weights=weights.numpy()
predecessor_chains=[]
predecessors_chains=[]
term_chains=[]
for i,bucket1 in enumerate(common_bucket_names):
  for j,bucket2 in enumerate(common_bucket_names):
    predecessor_chains.append((float(weights[j][i]),bucket1,bucket2))
    predecessors_chains.append((float(weights[j][i+len(common_bucket_names)]),bucket1,bucket2))
for term in range(N_TERMS):
  for j,bucket2 in enumerate(common_bucket_names):
    term_chains.append((float(weights[j][term+2*len(common_bucket_names)]),term*2+2009,bucket2))
chamber_chains=[]
for j,bucket in enumerate(common_bucket_names):
    chamber_chains.append((float(weights[j][1+term+2*len(common_bucket_names)]),"House",bucket))
for j,bucket in enumerate(common_bucket_names):
    chamber_chains.append((float(weights[j][2+term+2*len(common_bucket_names)]),"Senate",bucket))

In [ ]:
for param in sorted(chamber_chains):
  print(param[0],param[1],"->",param[2])

In [ ]:
# Sklearn
import sklearn
import copy
import random
model = sklearn.linear_model.LogisticRegression(penalty="l1",solver="saga",multi_class="multinomial")

scaler = sklearn.preprocessing.StandardScaler()
data_shuffled = copy.deepcopy(data)
random.shuffle(data_shuffled)
inputs = [np.concatenate([e["predecessors"],e["predecessor"],e["term"]]) for e in data_shuffled]
outputs = [e["output"] for e in data_shuffled]
inputs= scaler.fit_transform(inputs)
divider = int(len(data)*0.8)
train_inputs,test_inputs = inputs[:divider], data_shuffled[divider:]
train_outputs,test_outputs = outputs[:divider], outputs[divider:]
model.fit(train_inputs,train_outputs)
test_pred = model.predict_proba(test_inputs) 